In [1]:
# Parameters for simulation

# Funds to use
in_funds = ['VFINX', 'PRIDX']
out_funds = 'VUSTX'
benchmark = 'VFINX'
risk_free_fund = '^IRX'

# Date range (set to None to use max)
from datetime import datetime
date_from = None
date_to = None

# Starting cash position
growth_of = 10000

In [2]:
import yfinance as yf

# Download historical data
data = {}
all_funds = in_funds + [out_funds, risk_free_fund]
for symbol in all_funds:
    print(symbol)
    yf_ticker = yf.Ticker(symbol)
    yf_data = yf_ticker.history(period='max')
    data[symbol] = yf_data
    
benchmark = yf.Ticker(benchmark).history(period='max')

VFINX
PRIDX
VUSTX
^IRX


In [3]:
from datetime import timedelta

# Select only month-end prices
monthend = {}
for symbol, df in data.items():
    day1 = list(df.index.month)
    day2 = list(df.index.month)
    day2.pop(0)
    day2.append(-1)
    days = zip(day1, day2)
    truth_vals = [False if x[0] == x[1] else True for x in days]
    df_month_end = df[truth_vals].copy()
    
    if date_from is not None:
        tmp_date_from = date_from - timedelta(days=210)
        df_month_end = df_month_end[df_month_end.index >= tmp_date_from]
    if date_to is not None:
        df_month_end = df_month_end[df_month_end.index <= date_to]        
    
    df_month_end.index = df_month_end.index.strftime('%Y-%m')
    monthend[symbol] = df_month_end

In [4]:
# Calculate momentum signals
import numpy

risk_free_rate = monthend[risk_free_fund]['Close'] / 12

# 1-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum1 = (((close[1:]/close[:-1])-1)*100)
    momentum1 = numpy.insert(momentum1, 0, 0)
    df['Momentum1'] = momentum1
    df['Momentum1'] = df['Momentum1'] - risk_free_rate
    
# 3-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum3 = (((close[3:]/close[:-3])-1)*100)
    momentum3 = numpy.insert(momentum3, 0, [0, 0, 0])
    df['Momentum3'] = momentum3
    df['Momentum3'] = df['Momentum3'] - (risk_free_rate.rolling(3).sum())
    
# 6-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum6 = (((close[6:]/close[:-6])-1)*100)
    momentum6 = numpy.insert(momentum6, 0, [0, 0, 0, 0, 0, 0])
    df['Momentum6'] = momentum6
    df['Momentum6'] = df['Momentum6'] - (risk_free_rate.rolling(6).sum())    
    
# ADM Signal
for symbol, df in monthend.items():
    momentums = numpy.array((df['Momentum1'], df['Momentum3'], df['Momentum6']))
    
    # Use the following equation to match paper
    df['ADMSignal'] = numpy.average(momentums, axis=0)
    # Use the following equation to match portfolio visualizer
    #df['ADMSignal'] = df['Momentum1'] * .33 + df['Momentum3'] * .33 + df['Momentum6'] * .34

In [5]:
# Align time-axis for all symbols
import pandas
merged = None
for symbol, df in monthend.items():
    if not symbol in (in_funds + [out_funds]):
        continue
    if merged is None:
        merged = pandas.DataFrame(df['ADMSignal'])
        merged[f'{symbol}-ADMSignal'] = merged['ADMSignal']
        merged = merged.drop(columns=['ADMSignal'])
    else:
        df2 = pandas.DataFrame(df['ADMSignal'])
        df2[f'{symbol}-ADMSignal'] = df2['ADMSignal']
        df2 = df2.drop(columns=['ADMSignal'])
        merged = merged.merge(df2, how='inner', on='Date')
time_aligned = merged.dropna()
time_aligned = time_aligned.drop(f'{out_funds}-ADMSignal', axis=1)

In [6]:
# Build portfolio
symbols = pandas.DataFrame(time_aligned.idxmax(axis=1), columns=['Symbol'])
symbols['Symbol'] = [x.split('-')[0] for x in symbols['Symbol']]
signal  = pandas.DataFrame(time_aligned.max(axis=1), columns=['Signal'])
in_signal = symbols.merge(signal, how='inner', on='Date')
for idx in in_signal[in_signal['Signal'] < 0].index:
    in_signal.loc[idx, 'Symbol'] = out_funds
portfolio = in_signal

# Add signal columns
portfolio = portfolio.merge(time_aligned, how='inner', on='Date')

In [7]:
# Compute portfolio returns & growth of
current_asset = None
value = growth_of
shares = 0
for date in portfolio.index:
    new_asset = portfolio.loc[date]['Symbol']
    if current_asset is not None:
        # sell the current asset
        new_value = monthend[current_asset].loc[date]['Close'] * shares
        shares = 0
        monthly_return = new_value / value - 1
        value = new_value
    else:
        monthly_return = 0.0
    old_asset = current_asset
    current_asset = new_asset
    shares = value / monthend[new_asset].loc[date]['Close']
    portfolio.loc[date, 'MonthlyReturn'] = monthly_return
    portfolio.loc[date, 'Value'] = value
    portfolio.loc[date, 'NumShares'] = shares
    portfolio.loc[date, 'Price'] = monthend[new_asset].loc[date]['Close']
    if old_asset is not None:
        portfolio.loc[date, 'SoldPrice'] = monthend[old_asset].loc[date]['Close']
    else:
        portfolio.loc[date, 'SoldPrice'] = pandas.NA

In [8]:
# Update dates so they reflect the month that the asset should be owned
import pandas
dates = []
keep_dates = []
for dt in portfolio.index:
    yr, mo = map(int, dt.split('-'))
    if mo == 12:
        yr += 1
        mo = 1
    else:
        mo += 1
    dates.append((yr, mo))
    if date_from is not None:
        import calendar
        _, day = calendar.monthrange(yr, mo)
        dt = datetime(yr, mo, day)
        keep_dates.append((dt >= date_from) and (dt <= date_to))
    else:
        keep_dates.append(True)
portfolio.index = pandas.MultiIndex.from_tuples(dates, names=('Year', 'Month'))
keep_df = pandas.DataFrame(keep_dates)
keep_df.index = portfolio.index
portfolio = portfolio[keep_df[0]]

In [9]:
import calendar
from datetime import datetime

#  Compute daily returns for portfolio
returns = []
dates = []

# Step 1 get all trading days for the period that the portfolio is active
start_yr, start_mo = list(portfolio.index)[0]
start_date = datetime(start_yr, start_mo, 1)
end_yr, end_mo = list(portfolio.index)[-1]
_, end_day = calendar.monthrange(end_yr, end_mo)
end_date = datetime(end_yr, end_mo, end_day)

trading_days = []
for symbol, df in data.items():
    if list(df.index)[0] < start_date:
        for dt in list(df.index):
            if dt >= start_date and dt <= end_date:
                trading_days.append(dt)
        break
        
# Step 2 for each trading day compute portfolio return
trading_day = trading_days[0]
idx = (trading_day.year, trading_day.month)
asset = portfolio.loc[idx]['Symbol']
shares = growth_of / portfolio.loc[idx]['Price']
value = growth_of

for day in trading_days:
    # check if asset was sold
    idx = (day.year, day.month)
    if idx in list(portfolio.index):
        new_asset = portfolio.loc[idx]['Symbol']
        if new_asset != asset:
            value = portfolio.loc[idx]['SoldPrice'] * shares
            old_shares = shares
            shares = value / portfolio.loc[idx]['Price']
            old_asset = asset
            asset = new_asset
    
    current_close = data[asset].loc[day]['Close']
    new_value = shares * current_close
    ret = new_value / value - 1
    value = new_value
    row = {'Return': ret, 'Value': value, 'Asset': asset}
    returns.append(row)
    dates.append(day)
    
returns_df = pandas.DataFrame(returns, index=pandas.DatetimeIndex(dates, name='Date'))

In [10]:
# Print out last few items, to show what should be purchased
portfolio.tail(12)

Symbol     Signal  VFINX-ADMSignal  PRIDX-ADMSignal  MonthlyReturn  \
Year Month                                                                      
2019 12     VFINX   8.400887         8.400887         4.934418       0.036224   
2020 1      PRIDX   7.468355         7.180398         7.468355       0.030078   
     2      VFINX   4.852397         4.852397         2.928652      -0.017123   
     3      VUSTX  -0.380001        -4.375589        -0.380001      -0.082431   
     4      VUSTX -15.102111       -15.102111       -16.785935       0.056501   
     5      PRIDX   0.319519        -0.128895         0.319519       0.018686   
     6      PRIDX   6.678406         1.890366         6.678406       0.105195   
     7      PRIDX  13.070038         6.325886        13.070038       0.042303   
     8      PRIDX  13.719571         6.862086        13.719571       0.063839   
     9      PRIDX  17.870925        14.250359        17.870925       0.078289   
     10     PRIDX  20.887326        11.562066        20.887326      -0.007494   
     11     PRIDX  13.004732         5.704679        13.004732       0.002723   

                   Value      NumShares   Price SoldPrice  
Year Month                                                 
2019 12     2.510483e+06    8749.765529  286.92    286.92  
2020 1      2.585993e+06   37845.649085   68.33    295.55  
     2      2.541714e+06    8604.312094  295.40     67.16  
     3      2.332199e+06  158760.979788   14.69    271.05  
     4      2.463970e+06  158760.979788   15.52     15.52  
     5      2.510011e+06   40746.933286   61.60     15.81  
     6      2.774051e+06   40746.933286   68.08     68.08  
     7      2.891402e+06   40746.933286   70.96     70.96  
     8      3.075986e+06   40746.933286   75.49     75.49  
     9      3.316800e+06   40746.933286   81.40      81.4  
     10     3.291945e+06   40746.933286   80.79     80.79  
     11     3.300909e+06   40746.933286   81.01     81.01

In [11]:
# Compute benchmark returns
close = numpy.array(benchmark['Close'])
rets = (close[1:]/close[:-1]) - 1
rets = numpy.insert(rets, 0, 0)
benchmark['Returns'] = rets
benchmark_aligned = benchmark[benchmark.index > start_date]

In [12]:
import quantstats as qs
qs.reports.html(returns_df['Value'], benchmark, output='report.html')

In [13]:
from IPython.display import IFrame

IFrame(src='./report.html', width=1024, height=600)

In [14]:
portfolio.to_csv('portfolio.csv')